In [1]:
#container scaffolding
#updating to write all under capstone_j

In [1]:
#pip install cloudml-hypertune --user

Note: you may need to restart the kernel to use updated packages.


In [136]:
import hypertune 
import os

In [137]:
PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET = 'rutu-bucket'
REGION = "us-central1"


In [138]:
print(BUCKET)

rutu-bucket


In [139]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION


In [140]:
!echo $BUCKET

rutu-bucket


In [141]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi
gsutil ls -Lb gs://$BUCKET | grep "gs://\|Location"
echo $REGION

gs://rutu-bucket/ :
	Location type:			region
	Location constraint:		US-CENTRAL1
us-central1


In [142]:
%%bash
gcloud config set project ${PROJECT}
gcloud config set ai/region ${REGION}


Updated property [core/project].
Updated property [ai/region].


In [143]:
%%bash
mkdir -p trainer
touch trainer/__init__.py

In [144]:
!ls -R trainer/*

trainer/__init__.py  trainer/model.py  trainer/task.py


In [145]:
import os
print(os.getcwd())  # Prints the current working directory
#os.chdir('/home/jupyter/j-local-capstone')
#print(os.getcwd())  # Prints the current working directory

/home/jupyter/rutuja_april5


In [146]:
%%writefile trainer/task.py

import argparse
import json
import os
import sys
 

# getting the name of the directory
# where the this file is present.
current = os.path.dirname(os.path.realpath(__file__))

# Getting the parent directory name
# where the current directory is present.
parent = os.path.dirname(current)

# adding the parent directory to
# the sys.path.
sys.path.append(parent)
#import model
from trainer import model

import tensorflow as tf

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--train_data_path",
        help="GCS location of training data, e.g., gs://rutu-bucket/train_3d.csv",
        required=True
    )
    parser.add_argument(
        "--eval_data_path",
        help="GCS location of evaluation data, e.g., gs://rutu-bucket/valid_3d.csv",
        required=True
    )
    parser.add_argument(
        "--output_dir",
        help="GCS location to write checkpoints and export models",
        default=os.getenv("AIP_MODEL_DIR")
    )
    # parser.add_argument(
    #     "--batch_size",
    #     help="Number of examples to compute gradient over.",
    #     type=int,
    #     default=512
    # )
    # parser.add_argument(
    #     "--nnsize",
    #     help="Hidden layer sizes for DNN -- provide space-separated layers",
    #     default="64 64"
    # )
    parser.add_argument(
        "--num_epochs",
        help="Number of epochs to train the model.",
        type=int,
        default=40
    )
    # parser.add_argument(
    #     "--train_examples",
    #     help="""Number of examples (in thousands) to run the training job over.
    #     If this is more than actual # of examples available, it cycles through
    #     them. So specifying 1000 here when you have only 100k examples makes
    #     this 10 epochs.""",
    #     type=int,
    #     default=5000
    # )
    # parser.add_argument(
    #     "--eval_steps",
    #     help="""Positive number of steps for which to evaluate model. Default
    #     to None, which means to evaluate until input_fn raises an end-of-input
    #     exception""",
    #     type=int,
    #     default=None
    # )

    parser.add_argument(
        "--dropout_rate",
        help="""dropout_rate""",
        type=float,
        default=0.2)

    parser.add_argument(
        "--l2_regularization_lambda",
        help="""l2_regularization_lambda""",
        type=float,
        default=0.1)
    parser.add_argument("--hptune", action='store_true')  # if hptune is not set, it is False in default

    # Parse all arguments
    args = parser.parse_args()
    arguments = args.__dict__

    # Run the training job
    model.train_and_evaluate(arguments)


Overwriting trainer/task.py


In [148]:
%%writefile trainer/model.py

import os
import sys
import subprocess
import datetime
import pprint
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import hypertune 
from tensorflow.keras.utils import to_categorical
from io import BytesIO
from tensorflow.python.lib.io import file_io
from tensorflow.keras.layers import (
    Conv3D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling3D,
    Softmax
)

pp = pprint.PrettyPrinter(depth=8)
AIP_MODEL_DIR = os.environ["AIP_MODEL_DIR"]


labels_to_numeric = {
    "Arterial": 0,
    "Late": 1,
    "Non-Contrast": 2,
    "Venous": 3
}

numeric_to_labels = {
    0:   "Arterial",
    1:   "Late",    
    2:   "Non-Contrast",    
    3:  "Venous"
}


def reshape_and_normalize(images):
    
    ### START CODE HEREA

    # Reshape the images to add an extra dimension
    images = images.reshape((images.shape[0], images.shape[1], images.shape[2], images.shape[3],1))
    
    # Normalize pixel values
    max_value = np.max(images)
    images = images/max_value
    
    ### END CODE HERE

    return images,max_value# Reload the images in case you run this cell multiple times


def load_and_format_data_from_gcs(sample_dir):
    # sample_dir="gs://capstone-datasets/train_3d.csv"
    file_list = file_io.read_file_to_string(sample_dir).split("\n")
    images = np.array([np.load(BytesIO(file_io.read_file_to_string(file, binary_mode=True)))
                      for file in file_list if file])
    labels = np.array([os.path.basename(file).split("_")[4] for file in file_list if file])
    labels = np.array([labels_to_numeric[label] for label in labels])
    one_hots = to_categorical(labels)

    images_tranformed, max_value = reshape_and_normalize(images)
    return images_tranformed, one_hots

#training_sample_dir = "gs://rutu-bucket/train_3d.csv"
#training_images, one_hots = load_and_format_data_from_gcs(training_sample_dir)

class myCallback(tf.keras.callbacks.Callback):
    # Define the method that checks the accuracy at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') > 0.82:
            print("Reached 82% accuracy so cancelling training!")
            self.model.stop_training = True
        
            
            
#callbacks = myCallback()
def convolutional_model(training_images_shape=(32, 128, 128, 1)):
    ### START CODE HERE

    # Define the model
    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Conv3D(64, 3, activation='relu',input_shape=training_images_shape),
        tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),
        
        tf.keras.layers.Conv3D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),
        
        tf.keras.layers.Conv3D(128, 3, activation='relu'),
        tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),
        
        tf.keras.layers.Conv3D(256, 3, activation='relu'),
        tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),
        
        tf.keras.layers.GlobalAveragePooling3D(),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(rate=0.1),
        tf.keras.layers.Dense(4),
        tf.keras.layers.Softmax()      
    ])
    ### END CODE HERE

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
        
    return model

        
      
        
# Instantiate the HyperTune reporting object
hpt = hypertune.HyperTune()


# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag='val_loss',
            metric_value=logs['val_accuracy'],
            global_step=epoch)


def train_and_evaluate(args):

    training_images, one_hots_train = load_and_format_data_from_gcs(args["train_data_path"])
    print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
    print(f"Shape of training set after reshaping: {training_images.shape}\n")
    print(f"Shape of one image after reshaping: {training_images[0].shape}")

    valid_images, one_hots_valid = load_and_format_data_from_gcs(args["eval_data_path"])
    print(f"Maximum pixel value after normalization: {np.max(valid_images)}\n")
    print(f"Shape of training set after reshaping: {valid_images.shape}\n")
    print(f"Shape of one image after reshaping: {valid_images[0].shape}")

    model = convolutional_model(training_images_shape=training_images.shape[1:])
    print("Here is our model so far:\n")
    print(model.summary())

    #checkpoint_path = os.path.join(args["output_dir"], "checkpoints/phase_contrast")
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=AIP_MODEL_DIR, verbose=1, save_weights_only=True)

    callbacks = myCallback()

    history = model.fit(x=training_images, 
                    y=one_hots_train,
                    batch_size=16,
                    validation_data=(valid_images,one_hots_valid), 
                    epochs=40, 
                    callbacks=[callbacks, cp_callback, HPTCallback()])

    
    hptune = args["hptune"]
    history_history = history.history
    print(f"history_history: {pp.pformat(history_history)}")
    final_val_accuracy = history_history.get("val_accuracy")[-1]
    print(f"Final Validation accuracy: {final_val_accuracy}")
    if hptune:
        # Log it with hypertune
        hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag="final_val_accuracy", metric_value=final_val_accuracy
        )

    # Save the model #updated this j
    if not hptune:
        
        tf.saved_model.save(
            obj=model, export_dir=AIP_MODEL_DIR)  # with default serving function

    print("Exported trained model to {}".format(AIP_MODEL_DIR))
    
    
    
    
    
    
    #EXPORT_PATH = os.path.join(args["AIP_MODEL_DIR"], datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
    
    #tf.saved_model.save(obj=model, export_dir=EXPORT_PATH)  # with default serving function
    #tf.saved_model.save(obj=model, export_dir=AIP_MODEL_DIR)  # with default serving function

    #print("Exported trained model to {}".format(EXPORT_PATH))

Overwriting trainer/model.py


In [149]:
%%bash
START=$(date +%s);
echo $START
#sleep 1; # Your stuff

#export PYTHONPATH=${PYTHONPATH}:${PWD}/capstone_j
echo $PYTHONPATH

1680874989



In [48]:
#!pwd
#!ls -l -R *

In [100]:
%%bash
START=$(date +%s);
echo $START
#sleep 1; # Your stuff

BUCKET=rutu-bucket
OUTDIR=capstone1_trained_j_3_1
rm -rf ${OUTDIR}
export PYTHONPATH=${PYTHONPATH}:${PWD}
#PYTHONPATH='/home/jupyter/j-local-capstone/capstone_j/'
python3 -m trainer.task \
    --train_data_path=gs://${BUCKET}/train_3d.csv \
    --eval_data_path=gs://${BUCKET}/valid_3d.csv \
    --output_dir=${OUTDIR} \
    --num_epochs=1 

END=$(date +%s);
echo $END
echo $((END-START)) | awk '{print int($1/60)":"int($1%60)}'


1680804240
1680804253
0:13


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jupyter/rutuja_april5/trainer/task.py", line 20, in <module>
    from trainer import model
  File "/home/jupyter/rutuja_april5/trainer/model.py", line 23, in <module>
    AIP_MODEL_DIR = os.environ["AIP_MODEL_DIR"]
  File "/opt/conda/lib/python3.7/os.py", line 681, in __getitem__
    raise KeyError(key) from None
KeyError: 'AIP_MODEL_DIR'


In [150]:
%%writefile setup.py
from setuptools import find_packages
from setuptools import setup

setup(
    name='rutu_capstone_trainer_5_1',
    version='0.1',
    packages=find_packages(),
    include_package_data=True,
    description='CT contrast model training application2.'
)

Overwriting setup.py


In [151]:
%%bash
python ./setup.py sdist --formats=gztar



running sdist
running egg_info
writing rutu_capstone_trainer_5_1.egg-info/PKG-INFO
writing dependency_links to rutu_capstone_trainer_5_1.egg-info/dependency_links.txt
writing top-level names to rutu_capstone_trainer_5_1.egg-info/top_level.txt
reading manifest file 'rutu_capstone_trainer_5_1.egg-info/SOURCES.txt'
writing manifest file 'rutu_capstone_trainer_5_1.egg-info/SOURCES.txt'
running check
creating rutu_capstone_trainer_5_1-0.1
creating rutu_capstone_trainer_5_1-0.1/rutu_capstone_trainer_5_1.egg-info
creating rutu_capstone_trainer_5_1-0.1/trainer
copying files to rutu_capstone_trainer_5_1-0.1...
copying setup.py -> rutu_capstone_trainer_5_1-0.1
copying rutu_capstone_trainer_5_1.egg-info/PKG-INFO -> rutu_capstone_trainer_5_1-0.1/rutu_capstone_trainer_5_1.egg-info
copying rutu_capstone_trainer_5_1.egg-info/SOURCES.txt -> rutu_capstone_trainer_5_1-0.1/rutu_capstone_trainer_5_1.egg-info
copying rutu_capstone_trainer_5_1.egg-info/dependency_links.txt -> rutu_capstone_trainer_5_1-0.1/r

In [152]:
%%bash
BUCKET1=rutu-bucket
gsutil cp dist/rutu_capstone_trainer_5_1-0.1.tar.gz gs://${BUCKET1}/r-test/

Copying file://dist/rutu_capstone_trainer_5_1-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  3.5 KiB/  3.5 KiB]                                                
Operation completed over 1 objects/3.5 KiB.                                      


In [ ]:
#!gcloud ai custom-jobs describe projects/146339387885/locations/us-central1/customJobs/8920083161765904384

In [ ]:
#!gcloud ai custom-jobs stream-logs projects/146339387885/locations/us-central1/customJobs/8920083161765904384

In [ ]:
#!gcloud ai custom-jobs stream-logs projects/146339387885/locations/us-central1/customJobs/5353795206841892864

In [1]:
#!gsutil ls gs://qwiklabs-asl-04-76d2f9d7a5ae-jay/j-test/j-test/jay_capstone2_trainer-0.1.tar.gz
#!gsutil ls gs://qwiklabs-asl-04-76d2f9d7a5ae-jay/j-test/jay_capstone2_trainer-0.1.tar.gz


In [17]:
%%bash

BUCKET=rutu-bucket
#BUCKET1='qwiklabs-asl-04-76d2f9d7a5ae-jay'

TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
OUTDIR=gs://${BUCKET}/r-test/trained_model_$TIMESTAMP
JOB_NAME=rutu_ct_new_$TIMESTAMP


PYTHON_PACKAGE_URI=gs://${BUCKET}/r-test/rutu_capstone_trainer_5_1-0.1.tar.gz
#PYTHON_PACKAGE_URI='gs://qwiklabs-asl-04-76d2f9d7a5ae-jay/j-test/jay_capstone_trainer_5_1-0.1.tar.gz'
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest"
PYTHON_MODULE=trainer.task

echo > ./config.yaml "workerPoolSpecs:
  machineSpec:
    machineType: n1-highmem-16
    acceleratorType: NVIDIA_TESLA_P100
    acceleratorCount: 2
  replicaCount: 1
  pythonPackageSpec:
    executorImageUri: $PYTHON_PACKAGE_EXECUTOR_IMAGE_URI
    packageUris: $PYTHON_PACKAGE_URI
    pythonModule: $PYTHON_MODULE
    args:
    - --train_data_path=gs://${BUCKET}/train_3d.csv 
    - --eval_data_path=gs://${BUCKET}/valid_3d.csv
    - --output_dir=$OUTDIR
    - --num_epochs=10"

gcloud ai custom-jobs create \
  --region=${REGION} \
  --display-name=$JOB_NAME \
  --config=config.yaml

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/146339387885/locations/us-central1/customJobs/8546517489159241728] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/146339387885/locations/us-central1/customJobs/8546517489159241728

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/146339387885/locations/us-central1/customJobs/8546517489159241728


In [23]:
#!gcloud ai custom-jobs describe projects/146339387885/locations/us-central1/customJobs/3552359753940205568

In [24]:
#!gcloud ai custom-jobs stream-logs projects/146339387885/locations/us-central1/customJobs/3552359753940205568


## Hyperparameter tuning

To do hyperparameter tuning, create a YAML file and and pass its name with --config. This step could take hours -- you can increase --parallel-trial-count or reduce --max-trial-count to get it done faster. Since --parallel-trial-count is the number of initial seeds to start searching from, you don't want it to be too large; otherwise, all you have is a random search.

In [44]:
%%bash

BUCKET=capstone-datasets
BUCKET1='qwiklabs-asl-04-76d2f9d7a5ae-jay'
REGION='us-central1'
TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
#BASE_OUTPUT_DIR=gs://${BUCKET}/babyweight/hp_tuning_$TIMESTAMP
BASE_OUTPUT_DIR=gs://${BUCKET1}/jay-test/trained_model_$TIMESTAMP
JOB_NAME=jay_ct_hpt_$TIMESTAMP

#PYTHON_PACKAGE_URI=gs://${BUCKET}/babyweight/babyweight_trainer-0.1.tar.gz
PYTHON_PACKAGE_URI='gs://qwiklabs-asl-04-76d2f9d7a5ae-jay/j-test/jay_capstone_trainer_5_1-0.1.tar.gz'
#PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest"
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest"
PYTHON_MODULE=trainer.task

echo > ./hyperparam.yaml "displayName: $JOB_NAME
studySpec:
  metrics:
  - metricId: val_loss
    goal: MINIMIZE
  parameters:
  - parameterId: dropout_rate
    doubleValueSpec:
      minValue: 0.001
      maxValue: 0.4
    scaleType: UNIT_LOG_SCALE
  - parameterId: l2_regularization_lambda
    doubleValueSpec:
      minValue: 0.001
      maxValue: 0.4
    scaleType: UNIT_LOG_SCALE
  algorithm: ALGORITHM_UNSPECIFIED # results in Bayesian optimization
trialJobSpec:
  baseOutputDirectory:
    outputUriPrefix: $BASE_OUTPUT_DIR
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-highmem-16
      acceleratorType: NVIDIA_TESLA_P100
      acceleratorCount: 2
    pythonPackageSpec:
      executorImageUri: $PYTHON_PACKAGE_EXECUTOR_IMAGE_URI
      packageUris:
      - $PYTHON_PACKAGE_URI
      pythonModule: $PYTHON_MODULE
      args:
      - --train_data_path=gs://${BUCKET}/train_3d.csv 
      - --eval_data_path=gs://${BUCKET}/valid_3d.csv 
      - --num_epochs=40
    replicaCount: 1"
        
gcloud ai hp-tuning-jobs create \
    --region=$REGION \
    --display-name=$JOB_NAME \
    --config=hyperparam.yaml \
    --max-trial-count=20 \
    --parallel-trial-count=5

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Hyperparameter tuning job [192674431921815552] submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai hp-tuning-jobs describe 192674431921815552 --region=us-central1

Job State: JOB_STATE_PENDING


In [45]:
!gcloud ai custom-jobs describe projects/146339387885/locations/us-central1/customJobs/3552359753940205568

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
createTime: '2023-04-04T06:27:41.452284Z'
displayName: jay_ct_new_20230404_062740
endTime: '2023-04-04T06:35:02Z'
jobSpec:
  workerPoolSpecs:
  - diskSpec:
      bootDiskSizeGb: 100
      bootDiskType: pd-ssd
    machineSpec:
      acceleratorCount: 2
      acceleratorType: NVIDIA_TESLA_P100
      machineType: n1-highmem-16
    pythonPackageSpec:
      args:
      - --train_data_path=gs://capstone-datasets/train_3d.csv
      - --eval_data_path=gs://capstone-datasets/valid_3d.csv
      - --output_dir=gs://qwiklabs-asl-04-76d2f9d7a5ae-jay/jay-test/trained_model_20230404_062740
      - --num_epochs=10
      executorImageUri: us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest
      packageUris:
      - gs://qwiklabs-asl-04-76d2f9d7a5ae-jay/j-test/jay_capstone_trainer_5_1-0.1.tar.gz
      pythonModule: trainer.task
    replicaCount: '1'
name: projects/146339387885/locations/us-central1/customJobs/3552359753940205568
startTi

## Repeat training
This time with tuned parameters for batch_size and nembeds. Note that your best results may differ from below. So be sure to fill yours in!

In [ ]:
%%bash
TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
OUTDIR=gs://${BUCKET}/babyweight/tuned_$TIMESTAMP
JOB_NAME=babyweight_tuned_$TIMESTAMP

PYTHON_PACKAGE_URI=gs://${BUCKET}/babyweight/babyweight_trainer-0.1.tar.gz
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest"
PYTHON_MODULE=trainer.task

echo > ./tuned_config.yaml "workerPoolSpecs:
  machineSpec:
    machineType: n1-standard-8
  replicaCount: 1
  pythonPackageSpec:
    executorImageUri: $PYTHON_PACKAGE_EXECUTOR_IMAGE_URI
    packageUris: $PYTHON_PACKAGE_URI
    pythonModule: $PYTHON_MODULE
    args:
    - --train_data_path=gs://${BUCKET}/babyweight/data/train*.csv
    - --eval_data_path=gs://${BUCKET}/babyweight/data/eval*.csv
    - --output_dir=$OUTDIR
    - --num_epochs=10
    - --train_examples=20000
    - --eval_steps=100
    - --batch_size=32
    - --nembeds=8"
    
gcloud ai custom-jobs create \
  --region=${REGION} \
  --display-name=$JOB_NAME \
  --config=tuned_config.yaml

In [47]:
model = convolutional_model()

NameError: name 'convolutional_model' is not defined